In [1]:
import pandas as pd
import psycopg2
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.microsoft import EdgeChromiumDriverManager
import time

In [2]:
# connessione database
conn = psycopg2.connect(host='', database='', user='', password='')
c = conn.cursor()
c.execute('create table vini_tan(Annata text, Denominazione text, Vitigni text, Alcol text, Allergeni text, Consumo_ideale text, Temperatura_servizio text, Abbinamenti text, Prezzo text)')

In [3]:
url = 'https://www.tannico.it/bianchi.html?formato%5B0%5D=0.75l&_gl=1*18z4i38*_up*MQ..*_ga*MTA0Nzg0NjE3OC4xNjcwMjQxNzc0*_ga_79MD0HKQ9T*MTY3MDI0MTc3Mi4xLjAuMTY3MDI0MTc3Mi4wLjAuMA..'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.62'}

In [4]:
brow = webdriver.Edge(EdgeChromiumDriverManager().install())
brow.get(url)

In [5]:
# estrazione codice completo
start = 0
for x in range(400):
    brow.execute_script(f'window.scrollTo({start}, {start + 800})')
    start += 800
    time.sleep(1.5)

In [7]:
soup = BeautifulSoup(brow.page_source, 'html.parser')
vini = soup.find_all('div', {'class': 'productItem__info'})

In [ ]:
# raccolta informazioni
status = 0
for v in vini:
    link = v.find('a').get('href')
    r = requests.Session().get(link, headers=headers)
    v_soup = BeautifulSoup(r.content, 'html.parser')

    prezzo = v_soup.find('span', {'class': 'new-price'}).text if v_soup.find('span', {'class': 'new-price'}) != None else ''
    annata = v_soup.find('strong', string='Annata: ').next_sibling.text.strip() if v_soup.find('strong', string='Annata: ') != None else ''
    denominazione = v_soup.find('strong', string='Denominazione: ').next_sibling.text.strip() if v_soup.find('strong', string='Denominazione: ') != None else ''
    vitigni = v_soup.find('strong', string='Vitigni: ').next_sibling.text.strip() if v_soup.find('strong', string='Vitigni: ') != None else ''
    alcol = v_soup.find('strong', string='Alcol: ').next_sibling.text.strip() if v_soup.find('strong', string='Alcol: ') != None else ''
    allergeni = v_soup.find('strong', string='Allergeni: ').next_sibling.text.strip() if v_soup.find('strong', string='Allergeni: ') != None else ''
    consumo = v_soup.find('strong', string='Consumo ideale: ').next_sibling.text.strip() if v_soup.find('strong', string='Consumo ideale: ') != None else ''
    temperatura = v_soup.find('strong', string='Temperatura di servizio: ').next_sibling.text.strip() if v_soup.find('strong', string='Temperatura di servizio: ') != None else ''
    abbinamenti = v_soup.find('strong', string='Abbinamenti: ').next_sibling.text.strip() if v_soup.find('strong', string='Abbinamenti: ') != None else ''
    
    c.execute('insert into vini_tan(Annata, Denominazione, Vitigni, Alcol, Allergeni, Consumo_ideale, Temperatura_servizio, Abbinamenti, Prezzo) values(%s, %s, %s, %s, %s, %s, %s, %s, %s)',
              (annata, denominazione, vitigni, alcol, allergeni, consumo, temperatura, abbinamenti, prezzo))
    
    status += 1
    print(status)
    
    time.sleep(0.1)

conn.commit()

### Dati grezzi

In [11]:
df_vini_tan = pd.read_sql_query('select * from vini_tan', conn)
df_vini_tan.sample(3)

c:\Users\Ludo1\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,annata,denominazione,vitigni,alcol,allergeni,consumo_ideale,temperatura_servizio,abbinamenti,prezzo
580,2021,Alto Adige DOC,gewürztraminer 100%,13.5%,Solfiti,2022/2024,8/10 °C,"Antipasti di pesce, Piatti vegetariani, Second...","12,50 €"
1164,2019,Vigneti delle Dolomiti IGT,"chardonnay, riesling renano, sauvignon, kerner...",13.5%,Solfiti,2021/2027,10/12 °C,Primi di terra,"18,50 €"
319,2021,Friuli Colli Orientali DOC,friulano 100%,13%,Solfiti,2022/2024,8/10 °C,"Salumi, Zuppe","10,90 €"


### Dati riordinati

In [13]:
df_bianchi = pd.read_sql_query('select * from bianchi', conn)
df_bianchi.sample(3)

c:\Users\Ludo1\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,annata,denominazione,vitigni,alcol,consumo_ideale,temperatura_servizio,abbinamenti,prezzo
103,2021.0,igt,altri,12.5,2022/2024,13.0,antipasti,12.6
560,2019.0,doc,pinot,13.0,2021/2024,16.0,primi,18.2
841,2021.0,doc,pinot,13.5,2023/2025,13.0,altri,13.9


In [14]:
conn.close()